In [1]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/Spam classification/SPAM text message 20170820 - Data.csv")

In [2]:
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
data.isnull().sum()

Category    0
Message     0
dtype: int64

In [10]:
data.describe()

,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


In [4]:
# Data Preprocessing

import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
stop_words = stopwords.words('english')

corpus = []
for i in range(0,len(data)):
  msg = re.sub('[^a-zA-Z]',' ',data['Message'][i])
  msg = msg.lower()
  msg = msg.split()
  msg = [ps.stem(word) for word in msg if word not in stop_words]
  msg = ' '.join(msg)
  corpus.append(msg)

In [8]:
corpus[1]

'ok lar joke wif u oni'

In [9]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [11]:
#one hot representation
onehot = [one_hot(words,1500)for words in corpus]

In [12]:
onehot[1]

[48, 449, 248, 1106, 1321, 492]

In [13]:
# padding
embedded_docs = pad_sequences(onehot,padding='pre')

In [14]:
embedded_docs

array([[   0,    0,    0, ..., 1107, 1283, 1025],
       [   0,    0,    0, ..., 1106, 1321,  492],
       [   0,    0,    0, ..., 1375, 1424,  862],
       ...,
       [   0,    0,    0, ...,   50,   92,  890],
       [   0,    0,    0, ...,  559,  456,  593],
       [   0,    0,    0, ...,  871,  322, 1140]], dtype=int32)

In [16]:
embedding_vector_features = 30
model = Sequential()
model.add(Embedding(1500,embedding_vector_features,input_length = None))
model.add(Dropout(0.5))
model.add(LSTM(units = 200))
model.add(Dropout(0.5))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 30)          45000     
                                                                 
 dropout_2 (Dropout)         (None, None, 30)          0         
                                                                 
 lstm_1 (LSTM)               (None, 200)               184800    
                                                                 
 dropout_3 (Dropout)         (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 230,001
Trainable params: 230,001
Non-trainable params: 0
_________________________________________________________________


In [20]:
import numpy as np
x = np.array(embedded_docs)
y = data['Category']
y = y.replace('ham',0)
y = y.replace('spam',1)
y = np.array(y)

In [21]:
y

array([0, 0, 1, ..., 0, 0, 0])

In [22]:
x

array([[   0,    0,    0, ..., 1107, 1283, 1025],
       [   0,    0,    0, ..., 1106, 1321,  492],
       [   0,    0,    0, ..., 1375, 1424,  862],
       ...,
       [   0,    0,    0, ...,   50,   92,  890],
       [   0,    0,    0, ...,  559,  456,  593],
       [   0,    0,    0, ...,  871,  322, 1140]], dtype=int32)

In [23]:
from sklearn.model_selection import train_test_split

x_train , x_test, y_train , y_test = train_test_split(x,y,test_size = 0.20)

In [30]:
y_train

array([0, 0, 0, ..., 0, 0, 1])

In [24]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs = 10, batch_size = 64)

Epoch 1/10
70/70 [==============================] - 24s 301ms/step - loss: 0.3132 - accuracy: 0.8824 - val_loss: 0.1652 - val_accuracy: 0.9426
Epoch 2/10
70/70 [==============================] - 22s 314ms/step - loss: 0.0892 - accuracy: 0.9717 - val_loss: 0.0914 - val_accuracy: 0.9695
Epoch 3/10
70/70 [==============================] - 21s 294ms/step - loss: 0.0488 - accuracy: 0.9868 - val_loss: 0.0973 - val_accuracy: 0.9686
Epoch 4/10
70/70 [==============================] - 20s 291ms/step - loss: 0.0380 - accuracy: 0.9881 - val_loss: 0.0944 - val_accuracy: 0.9713
Epoch 5/10
70/70 [==============================] - 21s 299ms/step - loss: 0.0284 - accuracy: 0.9930 - val_loss: 0.1168 - val_accuracy: 0.9713
Epoch 6/10
70/70 [==============================] - 21s 303ms/step - loss: 0.0196 - accuracy: 0.9951 - val_loss: 0.1143 - val_accuracy: 0.9749
Epoch 7/10
70/70 [==============================] - 20s 292ms/step - loss: 0.0175 - accuracy: 0.9946 - val_loss: 0.1065 - val_accuracy: 0.9722

In [25]:
y_pred = model.predict(x_test)

In [37]:
y_pred = np.where(y_pred>0.5,1,0)

In [38]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_test,y_pred)

In [39]:
score

0.9757847533632287